---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    #import matplotlib.pyplot as plt
    #%matplotlib notebook
    
    
    # Your Code Here
    # PA:noi local clustering
    # SW_L 
    # SW_H shorter path vähemmän LCC
    
    i = 1
    
    for graph in P1_Graphs:
        
        #print("graph: " + str(i))
        i += 1
        
        #nx.draw_networkx(graph)
        sp = nx.average_shortest_path_length(graph)
        lcc = nx.average_clustering(graph)
        
        # Degree distr.
        degrees = graph.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes(graph)) for i in degree_values]
        
        #plt.figure()
        #plt.plot(histogram)
        # PA = in accordance with the power law . => 1st and 4th are PA
        # 2nd and 3rd are SW_L (large clustering)
        # Last one SW_H
        
        #print("avg sp: ", sp)
        #print("avg lcc", lcc)
    
    res = ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
    
    return res# Your Answer Here


graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [3]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

print(G.is_directed(), G.is_multigraph())  # undirected, not multigr

G.nodes(data=True)

#nx.periphery(G)  # Graph not connected: infinite path length
#nx.center(G) 

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458
False False


[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0}),
 (10, {'Department': 9, 'ManagementSalary': 0.0}),
 (11, {'Department': 14, 'ManagementSalary': 0.0}),
 (12, {'Department': 14, 'ManagementSalary': 1.0}),
 (13, {'Department': 26, 'ManagementSalary': 1.0}),
 (14, {'Department': 4, 'ManagementSalary': nan}),
 (15, {'Department': 17, 'ManagementSalary': 0.0}),
 (16, {'Department': 34, 'ManagementSalary': 0.0}),
 (17, {'Department': 1, 'ManagementSalary': 0.0}),
 (18, {'Department': 1, 'ManagementSalary': nan}),
 (19, {'Department': 14, 'Ma

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    # Your Code Here
    # test set is those missing. Train is those that have the value
    
    #from xgboost import XGBClassifier   
    
    from sklearn.ensemble import RandomForestClassifier
    
    df = pd.DataFrame(index=G.nodes())
    df.index.name = "Person"
    #print(df.head(20))
    
    df["Department"] = pd.Series(nx.get_node_attributes(G, "Department"))
    df["ManagementSalary"] = pd.Series(nx.get_node_attributes(G, "ManagementSalary"))
    
    # add other features. Degrees
    df['clustering'] = pd.Series(nx.clustering(G))   # Compute the clustering coefficient for nodes.
    # "Fraction of friends that are friend with each other"
    
    df['degree'] = pd.Series(G.degree())  # how many links
    
    ###df['eccentricity'] = pd.Series(nx.eccentricity(G))   # not connected, does not work
    ###df["avgShortestPath"] = 
    df["closenessCentrality"] = pd.Series(nx.closeness_centrality(G))
    df["betweennessCentrality"] = pd.Series(nx.betweenness_centrality(G))
    
    
    df["pagerank"] = pd.Series(nx.pagerank(G))
    
    #### hubs and auth dicts
    hubs, auth = nx.hits(G)
    df["hubScore"] = pd.Series(hubs)
    df["authScore"] = pd.Series(auth)
    
    # Nämä riittänee
    
    # subset of the missing ManagementSalary
    
    X_train = df[~pd.isnull(df["ManagementSalary"])]
    y_train = X_train["ManagementSalary"]
    X_train.drop("ManagementSalary", axis=1, inplace=True)
    
    X_test = df[pd.isnull(df["ManagementSalary"])]
    X_test.drop("ManagementSalary", axis=1, inplace=True)
    
    
    #print(X_train.head())
    
    #print(X_train.shape)  
    
    #print(X_test.head())
    
    
    #print(y_train.head())
    
    # Split train-test
    # Ok
    
    ### Train RF model (CV?)
    #clf = RandomForestClassifier().fit(X_train, y_train)
    
    
    # Try CV
    from sklearn.model_selection import RandomizedSearchCV
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1500, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 100, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    # # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, 
                                   verbose=0, random_state=42, n_jobs = -1)
    # verbose 0 no msgs
    # Fit the random search model
    rf_random.fit(X_train, y_train)
    
    # Predict_probas
    #pred_probas = clf.predict_proba(X_test)[:,1]
    pred_probas = rf_random.predict_proba(X_test)[:,1]
    
    #print(len(pred_probas))
    #print(pred_probas)
    
    pred = pd.Series(data=pred_probas, index=X_test.index)
    
    #print(pred_probas)
    
    
    return pred   #df.head(25)


salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Person
1       0.064444
2       0.964222
5       0.998222
8       0.108556
14      0.068667
18      0.097889
27      0.154333
30      0.765444
31      0.496556
34      0.072667
37      0.011333
40      0.064667
45      0.023556
54      0.546778
55      0.717778
60      0.477111
62      1.000000
65      0.974778
77      0.027667
79      0.069444
97      0.000000
101     0.000333
103     0.504667
108     0.103222
113     0.088222
122     0.010556
141     0.182889
142     0.997000
144     0.013667
145     0.523333
          ...   
913     0.000000
914     0.001333
915     0.000000
918     0.161111
923     0.002000
926     0.003000
931     0.000000
934     0.000000
939     0.000000
944     0.000000
945     0.000000
947     0.374627
950     0.069556
951     0.000000
953     0.000000
959     0.000000
962     0.000000
963     0.416889
968     0.000093
969     0.000582
974     0.000667
984     0.000000
987     0.014333
989     0.000427
991     0.001038
992     0.000000
994     0.000333
996    

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
# Dict of functions for converting values in certain columns. Keys can either
#    be integers or column labels


future_connections.head(10)


#sum(future_connections.isnull())
#future_connections.describe()

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
#pd.isnull(future_connections).count()  # 488 446
pd.isnull(future_connections).sum()

future_connections.index

Index([  (6, 840),   (4, 197), (620, 979), (519, 872), (382, 423),  (97, 226),
       (349, 905), (429, 860), (309, 989), (468, 880),
       ...
       (144, 824), (742, 985), (506, 684), (505, 916), (149, 214), (165, 923),
       (673, 755), (939, 940), (555, 905),  (75, 101)],
      dtype='object', length=488446)

In [6]:
#import operator

# This seemed too heavy / memory inefficient to go thru
def new_connections_predictions_old():

    
    #print(future_connections.head())
    
    
    #df = pd.DataFrame(data=future_connections, index=G.edges())
    df = future_connections.copy()
    
    #print(df.head())
    
    df.index.name = "Edge"
    #print(df.head(20))
    
    # Set community attrib
    for n in G.nodes():
        G.node[n]["community"] = G.node[n]["Department"]
    
    # cn_soundarajan_hopcroft
    cn_sh = list(nx.cn_soundarajan_hopcroft(G))
    df["CN_soundarajan_hopcroft_score"] = pd.Series(data=[k for i,j,k in cn_sh], index=[(i,j) for i,j,k in cn_sh])
    
    #print(df.head())
        
        
    ra_sh = list(nx.ra_index_soundarajan_hopcroft(G))
    df["RA_soundarajan_hopcroft_score"] = pd.Series(data=[k for i,j,k in ra_sh], index=[(i,j) for i,j,k in ra_sh])
    #print(df.head())
    
    # add features
    
    # Preferential attachment
    pref_attch = list(nx.preferential_attachment(G))
    df["pref_attachment_score"] = pd.Series(data=[k for i,j,k in pref_attch], index=[(i,j) for i,j,k in pref_attch])
    #print(df.head())
    
    # common neighbors between non-conected nodes
    # nbr of common neighbors for nodes that are not connected (there is no edge between them)
    commonNeigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
    #sorted(commonNeigh, key=operator.itemgetter(2), reverse=True)
    
    commonNeigh_series = pd.Series(data=[e[2] for e in commonNeigh], index=[(e[0],e[1]) for e in commonNeigh])
    df["nbrCommonNeighbors"] = commonNeigh_series
    #print(df.head())
    
    # JAccard coeff: common neighs normalized by neighbors
    # gives for non-connected
    jaccards = list(nx.jaccard_coefficient(G))
    #jaccards = [(e[0], e[1], e[2]) for e in jaccards]
    # not working
    #print(jaccards[:10])
    
    
    
    jaccards_series = pd.Series(data=[k for i,j,k in jaccards], index=[(i,j) for i,j,k in jaccards])
    #print(df.head())
    #df["jaccard_coeff"] = jaccards_series
    

    # resource alloc index
    #resAlloc = list(nx.resource_allocation_index(G)) # returns piter : iterator
    #An iterator of 3-tuples in the form (u, v, p) where (u, v) is a
    #pair of nodes and p is their resource allocation index.
    
    #df["resAllocIndex"] = pd.Series(data=[k for i,j,k in resAlloc], index=[(i,j) for i,j,k in resAlloc])
    
    #print(df.head())
    
    
    # train test split
    X_train = df[~pd.isnull(df["Future Connection"])]
    y_train = X_train["Future Connection"]
    X_train = X_train.drop("Future Connection", axis=1)
    
    X_test = df[pd.isnull(df["Future Connection"])]
    X_test = X_test.drop("Future Connection", axis=1)
    
    #print("X_test shape: ", X_test.shape)
    
    
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import RandomizedSearchCV
    
    # CV
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 80, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    # # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1, cv = 3, 
                                   verbose=2, random_state=42, n_jobs = -1)
    
    # verbose 0 no msgs
    # Fit the random search model
    rf_random.fit(X_train, y_train)
    
    # Predict_probas

    pred_probas = rf_random.predict_proba(X_test)[:,1]  # proba for positive class
    
    #print(len(pred_probas))
    #print(pred_probas)
    
    pred = pd.Series(data=pred_probas, index=X_test.index)
    
    
    return pred #df.sample(50) # Your Answer Here

#c = new_connections_predictions()

#new_connections_predictions()

ValueError: Found input variables with inconsistent numbers of samples: [36634, 3664]

In [6]:
#c[:10]

# New more lightweight one:

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

# Try multilayer perceptron

def new_connections_predictions():
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    # set community attribute for hopcroft metrics
    for n in G.nodes():
        G.node[n]['community'] = G.node[n]['Department']
        
    
    # Create features
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]    
    
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    # indeksit samassa järjestyksessä?
    
    cn_sh = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_sh = pd.DataFrame(index=[(x[0], x[1]) for x in cn_sh])
    df_cn_sh["cn_soundarajan_hopcroft"] = [x[2] for x in cn_sh]
    df = df.join(df_cn_sh, how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(0)

    commonNeigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
    commonNeigh_series = pd.Series(data=[e[2] for e in commonNeigh], index=[(e[0],e[1]) for e in commonNeigh])
    df["nbrCommonNeighbors"] = commonNeigh_series   
    

    df = future_connections.join(df, how='outer')
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    
    
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 
                'jaccard_coefficient', 'nbrCommonNeighbors']
    
    
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    
    # Scaling
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Classifier, two hidden layers
    clf = MLPClassifier(hidden_layer_sizes = [12, 6], alpha = 5.5,
                       random_state = 0, solver='lbfgs', verbose=0)
    
    clf.fit(X_train_scaled, Y_train)
    
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    preds = pd.Series(data=test_proba, index=X_test.index)
    
    
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [preds[x] for x in target.index]
    
    return target['prob']   # returns the series

new_connections_predictions()


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.027894
(542, 751)    0.010974
(20, 426)     0.581456
(50, 989)     0.011027
(942, 986)    0.011084
(324, 857)    0.011020
(13, 710)     0.159784
(19, 271)     0.139161
(319, 878)    0.011045
(659, 707)    0.010979
(49, 843)     0.011062
(208, 893)    0.010992
(377, 469)    0.008898
(405, 999)    0.018511
(129, 740)    0.017182
(292, 618)    0.020523
(239, 689)    0.011049
(359, 373)    0.009734
(53, 523)     0.216901
(276, 984)    0.011059
(202, 997)    0.011070
(604, 619)    0.061927
(270, 911)    0.011048
(261, 481)    0.072275
(200, 450)    0.999841
(213, 634)    0.010957
(644, 735)    0.048767
(346, 553)    0.010825
(521, 738)    0.010570
(422, 953)    0.018429
                ...   
(672, 848)    0.011048
(28, 127)     0.929353
(202, 661)    0.010802
(54, 195)     0.999831
(295, 864)    0.011011
(814, 936)    0.010966
(839, 874)    0.011084
(139, 843)    0.011016
(461, 544)    0.010392
(68, 487)     0.010361
(622, 932)    0.010999
(504, 936)    0.017277
(479, 528) 